In [65]:
from urllib import parse
import pandas as pd
import json
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime, timedelta
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
def construct_url(max_position=None):
    params = {'f': 'tweets', 'q': 'from:TSM_Leffen'}
    
    if max_position is not None:
        params['max_position'] = max_position
    
    url_segments = ('https', 'twitter.com', '/i/search/timeline', '', parse.urlencode(params), '')
    return parse.urlunparse(url_segments)

In [4]:
def execute_search(url):
        try:
            headers = {
                'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.'
                              '86 Safari/537.36'
            }
            req = requests.get(url, headers=headers)
            data = json.loads(req.text)
            return data

        # If we get a ValueError exception due to a request timing out, we sleep for our error delay, then make
        # another attempt
        except Exception as e:
            sleep(5)
            return execute_search(url)

In [5]:
def parse_tweets(html_json):
    soup = BeautifulSoup(html_json, "html.parser")
    tweets = []
    for li in soup.find_all('li', class_='js-stream-item'):
        if 'data-item-id' not in li.attrs:
            continue
        
        tweet = {
            'id': li['data-item-id'],
            'text': None,
            'created_at': None
        }
        
        # grabs the text of the tweet
        tweet_text = li.find('p', class_='tweet-text')
        if tweet_text is not None:
            tweet['text'] = tweet_text.get_text()
        
        # grabs the date it was created
        tweet_date = li.find("span", class_="_timestamp")
        if tweet_date is not None:
            tweet['created_at'] = datetime.fromtimestamp(int(tweet_date['data-time']))
        
        tweets.append(tweet)
    
    return tweets
        

In [10]:
def get_tweets():
    url = construct_url()
    response = execute_search(url)
    min_tweet = None
    tweets = []
    while response is not None and response['items_html'] is not None:
        subsection_tweets = parse_tweets(response['items_html'])
        tweets.extend(subsection_tweets)
        pprint(subsection_tweets)
        
        if not subsection_tweets:
            break
        
        if min_tweet is None:
            min_tweet = subsection_tweets[0]

        max_tweet = subsection_tweets[-1]

        if min_tweet['id'] is not max_tweet['id']:
            if 'min_position' in response.keys():
                max_position = response['min_position']
            else:
                max_position = "TWEET-%s-%s" % (max_tweet['id'], min_tweet['id'])
            url = construct_url(max_position=max_position)
            response = execute_search(url)
    
    return tweets

In [11]:
tweets = get_tweets()
df = pd.DataFrame(data=tweets)

[{'created_at': datetime.datetime(2017, 11, 23, 14, 22, 41),
  'id': '933777845570203648',
  'text': 'Battlerite time\n'
          'watch the ultimate gamer genes adapt to yet another '
          'gamehttp://twitch.tv/lffn\xa0'},
 {'created_at': datetime.datetime(2017, 11, 23, 12, 22, 34),
  'id': '933747614499131394',
  'text': 'dreamhack practice! also trying Battlerite again tonight now that '
          'its out of ea.\n'
          'http://twitch.tv/lffn\xa0'},
 {'created_at': datetime.datetime(2017, 11, 22, 8, 30, 51),
  'id': '933326912830578693',
  'text': 'afternoon melee + persona 5?\nhttp://twitch.tv/lffn\xa0'},
 {'created_at': datetime.datetime(2017, 11, 22, 5, 3, 19),
  'id': '933274685151830016',
  'text': 'I was a degen yesterday but today i will not be!'},
 {'created_at': datetime.datetime(2017, 11, 21, 8, 38, 20),
  'id': '932966410480955392',
  'text': 'Can you stream all of persona 5 yet?'},
 {'created_at': datetime.datetime(2017, 11, 21, 5, 29, 42),
  'id': '932918939

[{'created_at': datetime.datetime(2017, 11, 9, 4, 3, 35),
  'id': '928548613185294336',
  'text': 'laptop just bluescreens every 5 minutes\n'
          'Even after reinstalling windows\n'
          'Wtf do I do'},
 {'created_at': datetime.datetime(2017, 11, 8, 22, 55, 44),
  'id': '928471140602748928',
  'text': 'hipic.twitter.com/NxH2CULQvv'},
 {'created_at': datetime.datetime(2017, 11, 8, 17, 31, 14),
  'id': '928389477856407552',
  'text': 'actually so sick :('},
 {'created_at': datetime.datetime(2017, 11, 8, 11, 38, 19),
  'id': '928300663678881793',
  'text': 'also sick btw'},
 {'created_at': datetime.datetime(2017, 11, 8, 6, 45, 26),
  'id': '928226957388603392',
  'text': 'kekkai sensen is a sequel right? how good is s1?'},
 {'created_at': datetime.datetime(2017, 11, 8, 6, 39, 2),
  'id': '928225346960453632',
  'text': '...no black clover is not worth watching.'},
 {'created_at': datetime.datetime(2017, 11, 8, 6, 34, 52),
  'id': '928224294492147712',
  'text': 'any anime worth

[{'created_at': datetime.datetime(2017, 11, 5, 20, 36, 35),
  'id': '927348957084246016',
  'text': 'I am here on mobile twitter browser to let you know I have somehow '
          'made it to WF of Summit.\n'
          'What a rush. Ggs'},
 {'created_at': datetime.datetime(2017, 11, 5, 17, 32, 59),
  'id': '927302753033928704',
  'text': 'I am here on mobile browser twitter to tell everyone that told me I '
          "shouldn't quit that they were right."},
 {'created_at': datetime.datetime(2017, 11, 5, 11, 51, 35),
  'id': '927216836734676992',
  'text': 'I may not have much left in the tank after everything thats '
          "happened, but I promise that I'll give everything that I can.\n"
          "Let's do this."},
 {'created_at': datetime.datetime(2017, 11, 5, 11, 44, 55),
  'id': '927215160388808704',
  'text': 'New day. \n'
          'Deleting twitter app cuz im getting triggered already LUL\n'
          'Cya all later.'},
 {'created_at': datetime.datetime(2017, 11, 5, 1, 7, 37

[{'created_at': datetime.datetime(2017, 11, 4, 2, 18, 52),
  'id': '926695222691512320',
  'text': 'surprised more didnt guess it since mafia almost always destroys '
          'town really quickly at summits\n'
          'rishi def shouldve guessed it lmfao'},
 {'created_at': datetime.datetime(2017, 11, 4, 2, 18, 2),
  'id': '926695010673737728',
  'text': 'this bamboozle happened to us when we played in sweden\n'
          'we actually realized it when we were down to 3.\n'
          'fun times lmfao'},
 {'created_at': datetime.datetime(2017, 11, 4, 2, 15, 5),
  'id': '926694268835467264',
  'text': 'lmfao get fucked twitch chat/reddit \n'
          'aint so easy is it\n'
          'almost thought rishi would realize since I told him about this yday '
          'LOL'},
 {'created_at': datetime.datetime(2017, 11, 4, 0, 2, 16),
  'id': '926660843328094209',
  'text': 'HOPEFULLY MANGO'},
 {'created_at': datetime.datetime(2017, 11, 3, 20, 33, 12),
  'id': '926608232415608833',
  'text': 

[{'created_at': datetime.datetime(2017, 10, 29, 20, 23, 20),
  'id': '924793807727661056',
  'text': 'People calling me wholesome now explains why yall fall for hbox '
          'LMAO'},
 {'created_at': datetime.datetime(2017, 10, 29, 20, 22, 8),
  'id': '924793507558150144',
  'text': 'I think going back to my roots and taking out my anger/hatred '
          'through melee is doing wonders'},
 {'created_at': datetime.datetime(2017, 10, 29, 20, 18, 13),
  'id': '924792519044620291',
  'text': 'Feels like my mental game has never been better honestly, really '
          'picked it up after GTX.\n'
          'Now to fuse it with amazing ingame play '},
 {'created_at': datetime.datetime(2017, 10, 29, 20, 12, 29),
  'id': '924791078175244293',
  'text': 'Stop playing wii u and lets team imo'},
 {'created_at': datetime.datetime(2017, 10, 29, 20, 4, 50),
  'id': '924789154172547072',
  'text': 'Proud of myself for playing solidly and keeping it together '
          'mentally despite having a

...

[{'created_at': datetime.datetime(2013, 5, 11, 20, 46, 52),
  'id': '333382736100356096',
  'text': "@slinkyfish @xD1x If I'm forced to be friends with people I don't "
          "like to compete I'd rather go somewhere else. Sorry for being all "
          'emo lol'},
 {'created_at': datetime.datetime(2013, 5, 11, 20, 45, 11),
  'id': '333382309380255744',
  'text': "@slinkyfish @xD1x Well, I'm okay with not being best friends with "
          "everyone at a tournament, but that doesn't seem to be acceptable."},
 {'created_at': datetime.datetime(2013, 5, 11, 20, 42, 6),
  'id': '333381536311287809',
  'text': "@slinkyfish @xD1x and people don't really like me either. \n"
          " I'll probably travel to the US for other things tho and maybe "
          'enter smash then'},
 {'created_at': datetime.datetime(2013, 5, 11, 20, 36, 49),
  'id': '333380205081145344',
  'text': '@slinkyfish @xD1x Too bad ;/ And its not really because of the ban '
          "thing, I have better things to 

[{'created_at': datetime.datetime(2013, 4, 29, 12, 52, 40),
  'id': '328914743660273664',
  'text': '@progducto @xZydin @xD1x However, fighting games are so varied that '
          'trying to compare it to RPS is very inefficent'},
 {'created_at': datetime.datetime(2013, 4, 29, 12, 52, 3),
  'id': '328914587174969346',
  'text': '@progducto @xZydin @xD1x Comparing the entire concept of grabs in '
          'smash to the tick throw mixup is just.... useless. Of course its an '
          'mixup'},
 {'created_at': datetime.datetime(2013, 4, 29, 12, 33, 3),
  'id': '328909805831991297',
  'text': "@progducto @xZydin @xD1x I'd also say that fighting games are in "
          'its core Yomi, not RPS. That to me describes it a lot better.'},
 {'created_at': datetime.datetime(2013, 4, 29, 12, 29, 55),
  'id': '328909017269280769',
  'text': "@progducto @xZydin @xD1x In smash throwing often isn't best used "
          "against blocking, neither is it in marvel. There's no reason to "
          '

[{'created_at': datetime.datetime(2013, 3, 26, 20, 9, 7),
  'id': '316703394117066752',
  'text': 'For anyone wondering, I am not attending BEAST, I am still banned '
          'and I have still quit smash. I did my best to apologize, time to '
          'move on.'},
 {'created_at': datetime.datetime(2013, 3, 21, 17, 5, 45),
  'id': '314845307143856129',
  'text': "Thanks for all the support guys, you're the best. I'm insanely "
          "tired, hopefully there won't be too much drama when I wake up... "
          ':)'},
 {'created_at': datetime.datetime(2013, 3, 21, 15, 33, 37),
  'id': '314822120121192448',
  'text': '@progducto Thanks man, always great to know theres people out there '
          'you can talk to ! :)'},
 {'created_at': datetime.datetime(2013, 3, 21, 15, 32, 59),
  'id': '314821961941405698',
  'text': "@wWwDazwa Thanks, it really wasn't easy. I tried several times and "
          "just couldn't finish it"},
 {'created_at': datetime.datetime(2013, 3, 21, 15, 23, 7),

[{'created_at': datetime.datetime(2013, 2, 25, 7, 25, 2),
  'id': '306016956270649344',
  'text': "I was about to write a statement about my ban, but I'm probably "
          'just gonna quit instead. Why am I supporting a scene that hates me? '
          '#ssbm'},
 {'created_at': datetime.datetime(2013, 2, 22, 10, 41, 38),
  'id': '304979269854064640',
  'text': '@progducto wow >_>'},
 {'created_at': datetime.datetime(2013, 2, 18, 10, 11, 48),
  'id': '303522209681514496',
  'text': "Sorry for the delayed critiques. I'm going through some really "
          "serious family stuff right now, I'll look at them asap."},
 {'created_at': datetime.datetime(2013, 2, 16, 20, 16, 39),
  'id': '302949647004090368',
  'text': '@CloneHat I have NTSC at my home, since I use it to practice for '
          'American tournaments, and since it was hosted at my place we played '
          'it.'},
 {'created_at': datetime.datetime(2013, 2, 16, 19, 53, 49),
  'id': '302943901436739584',
  'text': 'Won the

[{'created_at': datetime.datetime(2012, 6, 30, 17, 39, 57),
  'id': '219183518499082242',
  'text': 'http://instagram.com/p/Mg2LNUjEo9\xa0'},
 {'created_at': datetime.datetime(2012, 5, 7, 10, 23, 56),
  'id': '199504845428895744',
  'text': '@RetroZombieTV \nSup. Game?\n;D'}]
[]


In [15]:
df

,created_at,id,text
0,2017-11-23 14:22:41,933777845570203648,Battlerite time\nwatch the ultimate gamer gene...
1,2017-11-23 12:22:34,933747614499131394,dreamhack practice! also trying Battlerite aga...
2,2017-11-22 08:30:51,933326912830578693,afternoon melee + persona 5?\nhttp://twitch.tv...
3,2017-11-22 05:03:19,933274685151830016,I was a degen yesterday but today i will not be!
4,2017-11-21 08:38:20,932966410480955392,Can you stream all of persona 5 yet?
5,2017-11-21 05:29:42,932918939570589701,its snowing :o
6,2017-11-21 04:04:14,932897429346766848,leeeets fucking go! #TSMWIN
7,2017-11-20 16:02:53,932715895138512903,Nice two days in uppsala with the boys\nOn tra...
8,2017-11-19 16:48:31,932364992351006720,: D
9,2017-11-19 12:50:35,932305116077658115,I kinda fuck with the swedish winter now that ...


In [73]:
date_start = datetime(2017, 11, 24)
date_end = date_start - timedelta(1)

for ind in range(10):
    print(df['created_at'][ind])

daily_tweets = []
for ind, row in df[:10].iterrows():
    if row['created_at'] < date_start and row['created_at'] > date_end:
        daily_tweets.append(row['created_at'])
    else:
        pprint(daily_tweets)
        del daily_tweets[:]
        daily_tweets.append(row['created_at'])
        date_start -= timedelta(1)
        date_end -= timedelta(1)

2017-11-23 14:22:41
2017-11-23 12:22:34
2017-11-22 08:30:51
2017-11-22 05:03:19
2017-11-21 08:38:20
2017-11-21 05:29:42
2017-11-21 04:04:14
2017-11-20 16:02:53
2017-11-19 16:48:31
2017-11-19 12:50:35
[Timestamp('2017-11-23 14:22:41'), Timestamp('2017-11-23 12:22:34')]
[Timestamp('2017-11-22 08:30:51'), Timestamp('2017-11-22 05:03:19')]
[Timestamp('2017-11-21 08:38:20'),
 Timestamp('2017-11-21 05:29:42'),
 Timestamp('2017-11-21 04:04:14')]
[Timestamp('2017-11-20 16:02:53')]


In [52]:
vader = SentimentIntensityAnalyzer()
copy = df[:500]
for ind in range(500):
    if '\n' in df['text'][ind]:
        df.at['text', ind] = df['text'][ind].replace('\n', ' ')
    print(df['text'][ind])
    print(vader.polarity_scores(df['text'][ind]))
    print()

Battlerite time watch the ultimate gamer genes adapt to yet another gamehttp://twitch.tv/lffn 
{'pos': 0.0, 'neg': 0.0, 'compound': 0.0, 'neu': 1.0}

dreamhack practice! also trying Battlerite again tonight now that its out of ea. http://twitch.tv/lffn 
{'pos': 0.0, 'neg': 0.0, 'compound': 0.0, 'neu': 1.0}

afternoon melee + persona 5? http://twitch.tv/lffn 
{'pos': 0.0, 'neg': 0.0, 'compound': 0.0, 'neu': 1.0}

I was a degen yesterday but today i will not be!
{'pos': 0.0, 'neg': 0.0, 'compound': 0.0, 'neu': 1.0}

Can you stream all of persona 5 yet?
{'pos': 0.0, 'neg': 0.0, 'compound': 0.0, 'neu': 1.0}

its snowing :o
{'pos': 0.0, 'neg': 0.412, 'compound': -0.1027, 'neu': 0.588}

leeeets fucking go! #TSMWIN
{'pos': 0.0, 'neg': 0.0, 'compound': 0.0, 'neu': 1.0}

Nice two days in uppsala with the boys On train home now, stream is back tomorrow :)
{'pos': 0.279, 'neg': 0.0, 'compound': 0.7003, 'neu': 0.721}

: D
{'pos': 0.0, 'neg': 0.0, 'compound': 0.0, 'neu': 0.0}

I kinda fuck with the

irl streaming and filming after parties are definitely just the new paparazzi
{'pos': 0.351, 'neg': 0.0, 'compound': 0.6597, 'neu': 0.649}

2nd season is peanuts compared to later manga arcs
{'pos': 0.0, 'neg': 0.0, 'compound': 0.0, 'neu': 1.0}

its making me feel like a kid reading early bleach/naruto again
{'pos': 0.217, 'neg': 0.0, 'compound': 0.3612, 'neu': 0.783}

dude my hero academia somehow keeps getting better and better lets fucking GOOO
{'pos': 0.485, 'neg': 0.0, 'compound': 0.8555, 'neu': 0.515}

people would just see it as whining tldr - marth wins slightly in almost every area (neutral, punish, recovery, stages, ease of use, mu exp)
{'pos': 0.197, 'neg': 0.168, 'compound': 0.2263, 'neu': 0.635}

whether its through labbing the matchup more, finding a FD CP, or picking another matchup entirely, you are always responsible for your loss
{'pos': 0.093, 'neg': 0.093, 'compound': 0.0, 'neu': 0.813}

to be perfectly honest/clear tho knowing more about real life matchup stats is 